<a href="https://colab.research.google.com/github/valmirf/redes_neurais_ple/blob/master/RBF/RBF_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rede Neural de Base Radial (RBF)

As redes RBF são redes de alimentação direta (feedforward) consistindo de três camadas:


1.   **Camada de entrada**: propaga os estímulos
2.   **Camada escondida**: Unidades de processamento localmente sintonizáveis, utilizando mapeamento não linear.
3.   **Camada de saída**: Unidades de processamento lineares.


****

**O treinamento dessa rede ocorre de forma híbrida**, combinando aprendizagem não supervisionada (ANS) com a supervisionada(AS). Isso ocorre, pois em geral não se sabe quais saídas se desejam para a camada escondida. Sendo assim, a distribuição de trabalhos ocorre:
*   **ANS**: Treina a camada escondida definindo seus parâmetros livres (centros, larguras dos campos receptivos e pesos).
*   **AS**: Determina os valores dos pesos entre as camadas escondidas e de saída, considerando constantes os parâmetros já definidos.


****

**O aprendizado consiste em** determinar os valores para:
*   centro das funções de base radial,
*   largura das funções,
*   pesos da camada de saída.


Além disso, para cada neurônio da camada escondida, ele computa uma função de base radial.


Os passos necessários são:
1.   Utilizar um algoritmo ANS para encontrar os centros (protótipo para um cluster) das RBF;
2.   Utilizar métodos heurísticos para determinar a largura (área de influência de um cluster) de cada função;
3.   Utilizar um AS para determinar os pesos da camada de saída da rede.

1ª Etapa: Inicialização dos grupos com K-Means

In [3]:
!git clone https://github.com/valmirf/redes_neurais_ple.git

Cloning into 'redes_neurais_ple'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 59 (delta 12), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (59/59), done.


Definição da função de base radial

In [25]:
#função de base radial gaussiana
def rbfGaussiana(x, c, s):
    return np.exp(-1 / (2 * s**2) * (x-c)**2)

#função de cálculo da largura do campo receptivo em que se repete o mesmo valor pra todos os neurônios
def computeEqualStds(centers,k):
  dist = [np.sqrt(np.sum((c1 - c2) ** 2)) for c1 in centers for c2 in centers]
  dMax = np.max(dist)
  stds = np.repeat(dMax / np.sqrt(2 * k), k)
  return stds

2ª Etapa - Treinamento de uma Rede Neural

In [32]:
from sklearn.cluster import KMeans
import numpy as np

class RBFNet(object):
    """Implementation of a Radial Basis Function Network"""

    def __init__(self, k=3, attnumber=8, lr=0.01, epochs=100, rbf=rbfGaussiana, computeStds=computeEqualStds):
        self.k = k  # grupos ou numero de neuronios na camada escondida
        self.lr = lr # taxa de aprendizagem
        self.epochs = epochs  # número de iterações
        self.rbf = rbfGaussiana # função de base radial
        self.computeStds = computeEqualStds  #função de cálculo da largura do campo receptivo

        self.w = np.random.randn(self.k,attnumber)
        self.b = np.random.randn(1)

    def fit(self, X, y):
        self.stds = []
        #K-Means pra pegar os centros inicias 
        #1º parâmetro da rede RBF
        kmeans = KMeans(
            n_clusters=self.k, init='random',
            n_init=10, max_iter=300).fit(X)
        self.centers = kmeans.cluster_centers_
        #print('centers: ', self.centers)
        
        #Cálculo la dargura do campo receptivo
        #2º parâmetro da rede RBF
        self.stds = self.computeStds(self.centers,self.k)
        # training
        for epoch in range(self.epochs):
            for i in range(X.shape[0]):
                # forward pass
                #calcula a saída de cada neurônio da função de base radial
                phi = np.array([self.rbf(X[i], c, s) for c, s, in zip(self.centers, self.stds)])
                #calcula somatório do produto da saída da função de base radial e os pesos
                F = phi.T.dot(self.w)
                F = np.sum(F) + self.b
                #saída da rede
                out = 0 if F < 0 else 1
                
                #função de perda 
                loss = (y[i] - out).flatten() ** 2
                #print('Loss: {0:.2f}'.format(loss[0]))

                #cálculo do erro
                error = (y[i] - out).flatten()
                #atualização dos pesos
                self.w = self.w + self.lr * error * phi 
                self.b = self.b + self.lr * error

    #calcula saída da rede RBF com a rede treinada
    def predict(self, X):
        y_pred = []
        error = 0
        for i in range(X.shape[0]):
            a = np.array([self.rbf(X[i], c, s) for c, s, in zip(self.centers, self.stds)])
            F = a.T.dot(self.w)
            F = np.sum(F) + self.b
            out = 0 if F < 0 else 1
            y_pred.append(out)

        return np.array(y_pred)




```
# Isto está formatado como código
```

# Executando a rede neural RBF

In [33]:
# Neste código vou utilizar o pandas, framework amplamente utilizado pra lidar com dados
import pandas as pd
from sklearn.model_selection import train_test_split

#carrega a base de dados e retorna conjuntos de treinamento e teste
def load_data():
    url = 'redes_neurais_ple/Perceptron/diabetes.csv'
    df = pd.read_csv(url)
    #remove a ultima coluna (dados)
    data = df[df.columns[:-1]]
    #normaliza os dados
    normalized_data = (data - data.min()) / (data.max() - data.min())
    #retorna a última coluna (rótulos)
    labels = df[df.columns[-1]]
    #separa em conjunto de treinamento e teste com seus respectivos rótulos
    X_train, X_test, y_train, y_test = train_test_split(normalized_data, labels, test_size=0.2, random_state=0)
    
    return X_train, X_test, y_train, y_test

#chama função que carrega base de dados
training_inputs, test_inputs, training_labels, test_labels = load_data()

  
rbfnet = RBFNet(lr=1e-2, k=3, computeStds=computeEqualStds)
rbfnet.fit(training_inputs.values, training_labels.values)

y_pred = rbfnet.predict(test_inputs.values)

errorabs = abs(test_labels.values-y_pred)

print('error: ', np.sum(errorabs)/len(test_labels))

error:  0.2792207792207792


# Descrição Mini Projeto

Utilizando o código acima, modifique a última seção (Executando com Base de Dados) para que ele seja executado com a base de dados do arquivo train-mod.csv. Depois, modifique a função de base radial implementada (Gaussiana), para a Multiquadrática e a Multiquadrática Inversa e calcule a taxa de erro.


1- Calcular a quantidade de neurônios escondidos:

a) 3

b) 5

c) 7

d) 9

Qual foi a melhor configuração? Avaliaria um outro valor?

2- Utilizando a melhor configuração da questão anterior, calcular a taxa de erro usando uma das outras maneiras de retorno da largura do campo receptivo da função de base radial, em que cada neurônio possui sua própria largura. 


3- Calcular a taxa de erro combinando 2 funções de Base Radial e as duas maneiras de cálculo da largura do campo receptivo:

a) Gaussiana

b) Multiquadrática Inversa


Qual foi a melhor configuração?



DATA DE ENTREGA: 09/09/2020
